In [210]:
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import inspect
from sqlalchemy.engine import reflection

In [211]:
df = pd.read_csv('data/Stations.csv')

In [212]:
# df.head()

In [213]:
engine = create_engine("sqlite:///mta.db.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# session connection to db
session = Session(engine)

In [214]:
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['fareData', 'sqlite_sequence', 'stationLocationData', 'turnStileData']


### Read in Fares csv and Turnstiles Fares Sations csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [215]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfTwo = pd.read_csv('data/fares_190209.csv')

In [216]:
dfOne.head()

,UNIT,STATION,REMOTE,STATION,Station ID
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1
1,R094,ASTORIA BLVD,R094,ASTORIA BLVD & 31ST STREET,2
2,R093,30 AV,R093,30TH AVENUE & 31ST STREET,3
3,R092,BROADWAY,R092,BROADWAY & 31ST STREET,4
4,R091,36 AV,R091,36TH AVENUE & 31ST STREET,5


In [217]:
dfTwo.head()

,REMOTE,STATION,FF,SEN/DIS,7-D AFAS UNL,30-D AFAS/RMF UNL,JOINT RR TKT,7-D UNL,30-D UNL,14-D RFM UNL,...,PATH 2-T,AIRTRAIN FF,AIRTRAIN 30-D,AIRTRAIN 10-T,AIRTRAIN MTHLY,STUDENTS,NICE 2-T,CUNY-120,CUNY-60,
0,R001,WHITEHALL STREET,59035,2875,415,1067,255,23503,37820,0,...,0,1069,0,0,0,2968,0,568,0,...
1,R003,CYPRESS HILLS,2682,147,16,41,0,2333,1325,0,...,0,30,0,0,0,544,0,45,0,...
2,R004,75TH STREET & ELDERTS LANE,6865,433,61,138,1,4839,3680,0,...,0,51,0,0,0,2420,0,53,0,...
3,R005,85TH STREET & FOREST PKWAY,7614,562,67,186,2,5007,4155,0,...,0,71,0,0,0,1249,0,100,0,...
4,R006,WOODHAVEN BOULEVARD,7835,477,57,169,0,6838,4732,0,...,0,64,0,0,0,1542,0,113,0,...


#### Merge Both tables on remote column


In [218]:
fareData = pd.merge(dfOne, dfTwo, on='REMOTE',how="outer")

##### Set index to Station ID

In [219]:
fareData.set_index('Station ID', inplace=True)

In [220]:
# fareData.head(1)

##### Remove duplicate columns

In [221]:
fareData.drop(columns=[' STATION_x', ' STATION_y'], inplace=True)

In [222]:
# fareData.head(1)

#### Take updated fare data and add to sqllite DB

In [223]:
# fareData.to_sql('fareData', con=engine)

In [224]:
# check to see that db has both tables
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['fareData', 'sqlite_sequence', 'stationLocationData', 'turnStileData']


### Read in turnstiles_190209.csv and turnstiles_fares_stations2.csv to a Pandas DataFrame
#### Join both data frames on Remote column
#### Remove Duplicate columns
#### Send to Sqlite DB


In [225]:
dfOne = pd.read_csv('data/turnstiles_fares_stations2.csv')
dfThree = pd.read_csv('data/turnstiles_190209.csv')

In [226]:
dfOne.head(1)

,UNIT,STATION,REMOTE,STATION,Station ID
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1


In [227]:
dfThree.head(1)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2/2/2019,3:00:00,REGULAR,6930882,2350417


In [228]:
turnstileData = pd.merge(dfOne, dfThree, on='UNIT')

In [229]:
turnstileData.head()

,UNIT,STATION_x,REMOTE,STATION,Station ID,C/A,SCP,STATION_y,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793
1,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,7:00:00,REGULAR,10390156,12828909
2,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,11:00:00,REGULAR,10390285,12829287
3,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,15:00:00,REGULAR,10390512,12829923
4,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,19:00:00,REGULAR,10390738,12830584


In [230]:
turnstileData['ENTRIES DIFF'] = turnstileData['ENTRIES'].diff()
turnstileData.columns = turnstileData.columns.str.strip()
turnstileData['EXITS DIFF'] = turnstileData['EXITS'].diff()
turnstileData['TOTAL ACTIVITY'] = turnstileData['ENTRIES DIFF'] + turnstileData['EXITS DIFF']

In [231]:
turnstileData.head()

,UNIT,STATION_x,REMOTE,STATION,Station ID,C/A,SCP,STATION_y,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES DIFF,EXITS DIFF,TOTAL ACTIVITY
0,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793,NaN,NaN,NaN
1,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,7:00:00,REGULAR,10390156,12828909,10.0,116.0,126.0
2,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,11:00:00,REGULAR,10390285,12829287,129.0,378.0,507.0
3,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,15:00:00,REGULAR,10390512,12829923,227.0,636.0,863.0
4,R095,ASTORIA DITMARS,R095,DITMARS BLVD & 31ST STREET,1,R515,00-03-00,ASTORIA DITMARS,NQW,BMT,2/2/2019,19:00:00,REGULAR,10390738,12830584,226.0,661.0,887.0


In [232]:
turnstileData.drop(columns=['STATION_x', 'STATION_y'], inplace=True)

In [233]:
turnstileData.set_index('Station ID',inplace=True)
turnstileData.head()

,UNIT,REMOTE,STATION,C/A,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES DIFF,EXITS DIFF,TOTAL ACTIVITY
Station ID,,,,,,,,,,,,,,,
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793,NaN,NaN,NaN
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,7:00:00,REGULAR,10390156,12828909,10.0,116.0,126.0
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,11:00:00,REGULAR,10390285,12829287,129.0,378.0,507.0
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,15:00:00,REGULAR,10390512,12829923,227.0,636.0,863.0
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,19:00:00,REGULAR,10390738,12830584,226.0,661.0,887.0


In [234]:
times = ['3:00:00', '7:00:00', '11:00:00', '15:00:00', '19:00:00',
       '23:00:00']

In [235]:
turnstileData = turnstileData.loc[turnstileData['TIME'].isin(times)]

In [236]:
turnstileData.loc[(turnstileData['ENTRIES DIFF'] < 0) | (turnstileData['ENTRIES DIFF'] > 10000), 'ENTRIES DIFF'] = np.NaN


In [237]:
turnstileData.loc[(turnstileData['EXITS DIFF'] < 0) | (turnstileData['EXITS DIFF'] > 10000), 'EXITS DIFF'] = np.NaN


In [238]:
turnstileData.loc[(turnstileData['TOTAL ACTIVITY'] < 0) | (turnstileData['TOTAL ACTIVITY'] > 10000), 'TOTAL ACTIVITY'] = np.NaN


In [243]:
turnstileData.head(1)

,UNIT,REMOTE,STATION,C/A,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES DIFF,EXITS DIFF,TOTAL ACTIVITY
Station ID,,,,,,,,,,,,,,,
1,R095,R095,DITMARS BLVD & 31ST STREET,R515,00-03-00,NQW,BMT,2/2/2019,3:00:00,REGULAR,10390146,12828793,NaN,NaN,NaN


In [240]:
# turnstileData.to_sql('turnStileData', con=engine)

## Test queries

#### Fare data query testing


 "FF", "30-D UNL", "7-D UNL", "SEN/DIS", "7-D AFAS UNL", "30-D AFAS/RMF UNL",
    "JOINT RR TKT", "14-D RFM UNL", "1-D UNL", "14-D UNL", "7D-XBUS PASS", "TCMC",
    "RF 2 TRIP", "RR UNL NO TRADE", "TCMC ANNUAL MC", "MR EZPAY EXP", "MR EZPAY UNL", "PATH 2-T",
    "AIRTRAIN FF", "AIRTRAIN 30-D", "AIRTRAIN 10-T", "AIRTRAIN MTHLY", "STUDENTS", "NICE 2-T",
    "CUNY-120", "CUNY-60"

In [24]:
fareData = Base.classes.fareData

In [25]:
sel = [
        fareData.FF,
        fareData.SEN_DIS,
        fareData.SEVEN_D_AFAS_UNL,
        fareData.THIRTY_D_AFAS_RMF_UNL,
        fareData.JOINT_RR_TKT,
        fareData.SEVEN_D_UNL,
        fareData.THIRTY_D_UNL,
        fareData.FOURTEEN_D_RFM_UNL,
        fareData.ONE_D_UNL,
        fareData.FOURTEEN_D_UNL,
        fareData.SEVEND_XBUS_PASS,
        fareData.TCMC,
        fareData.RF_TWO_TRIP,
        fareData.RR_UNL_NO_TRADE,
        fareData.TCMC_ANNUAL_MC,
        fareData.MR_EZPAY_EXP,
        fareData.MR_EZPAY_UNL,
        fareData.PATH_TWO_T,
        fareData.AIRTRAIN_FF,
        fareData.AIRTRAIN_THIRTY_D,
        fareData.AIRTRAIN_TEN_T,
        fareData.AIRTRAIN_MTHLY,
        fareData.STUDENTS,
        fareData.NICE_TWO_T,
        fareData.CUNY_ONETWENTY,
        fareData.CUNY_SIXTY
    
    ]

In [26]:
print(session.query(*sel).filter(fareData.Station_ID == 1).all())

[(29208, 1596, 155, 263, 23, 12850, 29457, 0, 0, 0, 21, 403, 743, 16, 3585, 787, 814, 0, 251, 0, 0, 0, 2774, 0, 66, 0)]
